In [2]:
from transformers import ViTModel, ViTConfig
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from PIL import Image
from torchvision import transforms
import torch

/Users/ns/cpet-ai/CPET-AI/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import os

class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.startswith('heatmap_') and f.endswith('.png')]
        self.image_files.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))  # Sort by index number

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        # Extract index from filename (e.g., 2079 from heatmap_2079.png)
        index = int(self.image_files[idx].split('_')[1].split('.')[0])
        
        return image, index

Global mean: tensor([0.6479, 0.7233, 0.7550]), Global std: tensor([0.3835, 0.3325, 0.2662])


In [ ]:
# Image transformations
transform_raw = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # Resize to ViT input size
        transforms.ToTensor(),
        # No normalization here - we'll compute and apply global stats later
    ]
)

# We'll need to compute global mean and std across the dataset
def compute_global_stats(dataset):
    loader = DataLoader(dataset, batch_size=32, num_workers=0)
    mean = 0.
    std = 0.
    total_images = 0
    
    # Compute mean
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        total_images += batch_samples
    
    mean = mean / total_images
    
    # Compute std
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        std += ((images - mean.unsqueeze(1))**2).sum([0,2])
    
    std = torch.sqrt(std / (total_images * images.size(2)))
    
    return mean, std


# Load the dataset
dataset = ImageDataset(image_dir='/Users/ns/Downloads/heatmaps', transform=transform_raw)

# Compute global mean and std
mean, std = compute_global_stats(dataset)
print(f"Global mean: {mean}, Global std: {std}")


In [12]:
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # Resize to ViT input size
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.6479, 0.7233, 0.7550],  # These are precomputed global stats, recalculate if the datset changes
            std=[0.3835, 0.3325, 0.2662]  # # These are precomputed global stats, recalculate if the datset changes
        )
    ]
)

# Load pre-trained ViT model
config = ViTConfig.from_pretrained("google/vit-base-patch16-224")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224")

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Create dataset with normalization transform
dataset = ImageDataset(image_dir='/Users/ns/Downloads/heatmaps', transform=transform)

# Create dataloader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Extract features
features = []
vit_model.eval()  # Set model to evaluation mode

with torch.no_grad():
    for images, _ in dataloader:
        # Get model outputs
        outputs = vit_model(images)

        cls_features = outputs.last_hidden_state[:, 0, :]
        features.append(cls_features)

# Concatenate all features
features = torch.cat(features, dim=0).cpu().numpy()
features = [features[i] for i in range(len(features))]
print(f"Extracted features shape: {features.shape}")

AttributeError: 'list' object has no attribute 'shape'

In [19]:
# Save features to disk using numpy
import numpy as np
features = np.array(features)  # Convert list to numpy array
np.save('vit_features.npy', features)
print(f"Saved features with shape {features.shape} to vit_features.npy")


Saved features with shape (2089, 768) to vit_features.npy


In [50]:
from preprocess import preprocess_data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np

use_only_demographics = False  # Set to True to use only demographic features

# Load saved features
features = np.load('vit_features.npy')
print(f"Loaded features with shape {features.shape} from vit_features.npy")
df_interpolated, filtered_time_series_columns = preprocess_data('/Users/ns/Downloads/combined_outcome_df.parquet')
y = df_interpolated["composite_outcome"]

# Add demographic features to the feature arrays
demographic_columns = ['Gender', 'Age', 'Height', 'Weight', 'TestDuration', 
                      'ExerciseDuration', 'BarometricPress', 'AmbientTemp', 'AmbientRH']

# Convert Gender to numeric (0 for Female, 1 for Male)
df_interpolated['Gender'] = df_interpolated['Gender'].map({'Female': 0, 'Male': 1})

# Get demographic features from df_interpolated
demographic_features = df_interpolated[demographic_columns].values


if use_only_demographics:
    features = demographic_features
    print(f"Using only demographic features with shape: {features.shape}")
else:
    # Combine ViT features with demographic features
    #features = np.hstack([features, demographic_features])
    print(f"Combined features shape after adding demographics: {features.shape}")


Loaded features with shape (2089, 768) from vit_features.npy
Combined features shape after adding demographics: (2089, 768)


In [51]:
from sklearn.preprocessing import StandardScaler

# Initialize list to store AUC scores
auc_scores = []

# Run 100 iterations with different random splits
for i in range(5):
    # Split data into train/test sets, stratified by outcome
    X_train, X_test, y_train, y_test = train_test_split(
        features, y, 
        test_size=0.3,
        random_state=i,  # Different random state each iteration
        stratify=y,
        shuffle=True
    )

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train logistic regression with L1 regularization
    model = LogisticRegression(
        penalty='l1',
        solver='liblinear',
        random_state=i,
        C=1
    )
    model.fit(X_train_scaled, y_train)

    # Get prediction probabilities
    y_pred_proba = model.predict_proba(X_test_scaled)[:,1]

    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    auc_scores.append(roc_auc)

    # Print progress every 10 iterations
    if (i + 1) % 10 == 0:
        print(f"Completed {i + 1} iterations...")

# Calculate and print statistics
mean_auc = np.mean(auc_scores)
std_auc = np.std(auc_scores)
print(f"\nResults over {len(auc_scores)} runs:")
print(f"Mean ROC AUC: {mean_auc:.3f} ± {std_auc:.3f}")
print(f"Min ROC AUC: {min(auc_scores):.3f}")
print(f"Max ROC AUC: {max(auc_scores):.3f}")



Results over 5 runs:
Mean ROC AUC: 0.624 ± 0.022
Min ROC AUC: 0.595
Max ROC AUC: 0.652


In [52]:
from xgboost import XGBClassifier

# Initialize lists to store results
auc_scores = []

# Run multiple iterations
for i in range(10):
    # Split data into train/test sets, stratified by outcome 
    X_train, X_test, y_train, y_test = train_test_split(
        features, y,
        test_size=0.3, 
        random_state=i,  # Different random state each iteration
        stratify=y,
        shuffle=True
    )

    # Train XGBoost classifier
    model = XGBClassifier(
        max_depth=2,
        learning_rate=0.1,
        n_estimators=100,
        min_child_weight=1,
        gamma=0,
        subsample=0.5,
        colsample_bytree=0.5,
        random_state=i,
        use_label_encoder=False,
        eval_metric='logloss',
    )
    model.fit(X_train, y_train)

    # Get prediction probabilities
    y_pred_proba = model.predict_proba(X_test)[:,1]

    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    auc_scores.append(roc_auc)

    # Print progress every 10 iterations
    if (i + 1) % 10 == 0:
        print(f"Completed {i + 1} iterations...")

# Calculate and print statistics
mean_auc = np.mean(auc_scores)
std_auc = np.std(auc_scores)
print(f"\nResults over {len(auc_scores)} runs:")
print(f"Mean ROC AUC: {mean_auc:.3f} ± {std_auc:.3f}")
print(f"Min ROC AUC: {min(auc_scores):.3f}")
print(f"Max ROC AUC: {max(auc_scores):.3f}")


Completed 10 iterations...

Results over 10 runs:
Mean ROC AUC: 0.678 ± 0.025
Min ROC AUC: 0.638
Max ROC AUC: 0.716


In [15]:
import torch.nn as nn
import os
from preprocess import preprocess_data

# Load pre-trained ViT model
config = ViTConfig.from_pretrained("google/vit-base-patch16-224")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224")

# Freeze all weights except pooler
for name, param in vit_model.named_parameters():
    if 'pooler' not in name:  # Only pooler weights remain unfrozen
        param.requires_grad = False

transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # Resize to ViT input size
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.6479, 0.7233, 0.7550],  # These are precomputed global stats, recalculate if the datset changes
            std=[0.3835, 0.3325, 0.2662]  # # These are precomputed global stats, recalculate if the datset changes
        )
    ]
)

class ImageDatasetLabels(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.startswith('heatmap_') and f.endswith('.png')]
        self.image_files.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))  # Sort by index number
        df_interpolated, filtered_time_series_columns = preprocess_data('/Users/ns/Downloads/combined_outcome_df.parquet')
        self.labels = df_interpolated["composite_outcome"]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        # Extract index from filename (e.g., 2079 from heatmap_2079.png)
        index = int(self.image_files[idx].split('_')[1].split('.')[0])
        
        return image, torch.tensor(self.labels.iloc[idx])

# Create dataset with normalization transform
dataset = ImageDatasetLabels(image_dir='/Users/ns/Downloads/heatmaps', transform=transform)

# Split dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Use the built-in pooler
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, vit_model.parameters()), lr=1e-5)  # Only optimize unfrozen params
criterion = nn.BCEWithLogitsLoss()

# Training loop
num_epochs = 10
vit_model.train()

# Add classification head
classifier = nn.Linear(768, 1)  # Project from pooler_output dim to binary classification


for epoch in range(num_epochs):
    epoch_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Get ViT outputs - pooled_output is already available
        outputs = vit_model(images)
        pooled_output = outputs.pooler_output  # This is the built-in pooled representation
        
        # Add a simple projection to get logits
        logits = classifier(pooled_output).squeeze()
        loss = criterion(logits, labels.float())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

# Extract final features
train_features = []
test_features = []
vit_model.eval()

with torch.no_grad():
    # Extract training features
    for images, _ in train_loader:
        outputs = vit_model(images)
        pooled_features = outputs.pooler_output
        train_features.append(pooled_features)
    
    # Extract test features
    for images, _ in test_loader:
        outputs = vit_model(images)
        pooled_features = outputs.pooler_output
        test_features.append(pooled_features)

# Concatenate features
train_features = torch.cat(train_features, dim=0).cpu().numpy()
test_features = torch.cat(test_features, dim=0).cpu().numpy()
print(f"Training features shape: {train_features.shape}")
print(f"Test features shape: {test_features.shape}")

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10, Loss: 0.5033
Epoch 2/10, Loss: 0.4085
Epoch 3/10, Loss: 0.4013
Epoch 4/10, Loss: 0.3984
Epoch 5/10, Loss: 0.3873
Epoch 6/10, Loss: 0.3840
Epoch 7/10, Loss: 0.3815
Epoch 8/10, Loss: 0.3789
Epoch 9/10, Loss: 0.3780
Epoch 10/10, Loss: 0.3757
Training features shape: (1671, 768)
Test features shape: (418, 768)


In [18]:
# Save features to numpy files
import numpy as np
np.save('train_features.npy', train_features)
np.save('test_features.npy', test_features)

In [20]:
# Calculate ROC AUC on test set
from sklearn.metrics import roc_auc_score
# Extract test labels from test dataset
test_labels = []
for _, labels in test_loader:
    test_labels.append(labels)
test_labels = torch.cat(test_labels, dim=0).cpu().numpy()

# Get predictions on test set
test_logits = classifier(torch.from_numpy(test_features)).squeeze().detach().numpy()
test_preds = 1 / (1 + np.exp(-test_logits))  # Apply sigmoid to get probabilities

# Calculate ROC AUC
test_roc_auc = roc_auc_score(test_labels, test_preds)
print(f"Test ROC AUC: {test_roc_auc:.4f}")

Test ROC AUC: 0.6966
